In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pandas==1.2.4


     |████████████████████████████████| 9.9 MB 5.0 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.2.4 which is incompatible.


In [ ]:
!pip freeze

absl-py==0.12.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.1.0
appdirs==1.4.4
argon2-cffi==20.1.0
arviz==0.11.2
astor==0.8.1
astropy==4.2.1
astunparse==1.6.3
async-generator==1.10
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.2.0
audioread==2.1.9
autograd==1.3
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==3.3.0
blis==0.4.1
bokeh==2.3.3
Bottleneck==1.3.2
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.6
cached-property==1.5.2
cachetools==4.2.2
catalogue==1.0.0
certifi==2021.5.30
cffi==1.14.6
cftime==1.5.0
chardet==3.0.4
charset-normalizer==2.0.2
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorcet==2.0.6
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.3.2
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.2.6
cvxpy==1.0.31
cycler==0.10.0
cymem==2.0.5
Cython==0.29.23
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
dill==0.3.4
distributed==1.25.3
d

In [ ]:
!cp /content/drive/MyDrive/mlb/config.py .
!cp /content/drive/MyDrive/mlb/preprocessing.py .

In [ ]:
from preprocessing import *
from config import *

import gc
import pathlib

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
PATH_TO_MLB_DATA = BASE_DIR / "data/mlb-player-digital-engagement-forecasting"

In [ ]:
train = pd.read_csv(PATH_TO_MLB_DATA / "train.csv")

In [ ]:
# prepare training data

# dates and next day player enagagement
unpacked_next_day_player_engagement = unpack_data(train, ["nextDayPlayerEngagement"])["nextDayPlayerEngagement"]
del(train["nextDayPlayerEngagement"])
gc.collect()
seasons = pd.read_csv(PATH_TO_MLB_DATA / "seasons.csv")
dates_with_info = get_dates_with_info(unpacked_next_day_player_engagement, seasons)
del(seasons)
gc.collect()
train_df = unpacked_next_day_player_engagement.merge(dates_with_info, on="dailyDataDate", how="left")
del(dates_with_info)
gc.collect()

print("# lagged targets")
for lag in range(1, N_LAGS+1):
    lagged_targets = get_lagged_targets_train(unpacked_next_day_player_engagement, lag)
    train_df = train_df.merge(lagged_targets, on=["dailyDataDate", "playerId"], how="left")
    del(lagged_targets)
    gc.collect()
del(unpacked_next_day_player_engagement)
gc.collect

#print("# player target stats")
#player_target_stats = get_player_target_stats(unpacked_next_day_player_engagement)
#train_df = train_df.merge(player_target_stats, on="playerId", how="left")
#del(player_target_stats, unpacked_next_day_player_engagement)
#gc.collect

print("# players")
players = pd.read_csv(PATH_TO_MLB_DATA / "players.csv")
players_to_merge = prepare_players(players)
del(players)
gc.collect()
train_df = train_df.merge(players_to_merge, on="playerId", how="left")
del(players_to_merge)
gc.collect()

print("# rosters")
unpacked_rosters = unpack_data(train, ["rosters"])["rosters"]
del(train["rosters"])
gc.collect()
rosters_to_merge = prepare_rosters(unpacked_rosters)
del(unpacked_rosters)
gc.collect()
train_df = train_df.merge(rosters_to_merge, on=["dailyDataDate", "playerId"], how="left")
del(rosters_to_merge)
gc.collect()

print("# teams")
teams = pd.read_csv(PATH_TO_MLB_DATA / "teams.csv")
teams_to_merge = prepare_teams(teams)
del(teams)
gc.collect()
train_df = train_df.merge(teams_to_merge, on="teamId", how="left")
del(teams_to_merge)
gc.collect()

print("# games")
unpacked_games = unpack_data(train, ["games"])["games"]
del(train["games"])
gc.collect()
games_to_merge = prepare_games(unpacked_games)
train_df = train_df.merge(games_to_merge, on=["dailyDataDate", "teamId"], how="left")
del(games_to_merge)
gc.collect()

# team box scores
#unpacked_team_box_scores = unpack_data(train, ["teamBoxScores"])["teamBoxScores"]
#del(train["teamBoxScores"])
#gc.collect()
#team_box_scores_to_merge = prepare_team_box_scores(unpacked_team_box_scores)
#del(unpacked_team_box_scores)
#gc.collect()
#train_df = train_df.merge(team_box_scores_to_merge, on=["dailyDataDate", "teamId"], how="left")
#del(team_box_scores_to_merge)
#gc.collect()

print("# player box scores")
unpacked_player_box_scores = unpack_data(train, ["playerBoxScores"])["playerBoxScores"]
del(train["playerBoxScores"])
gc.collect()
player_box_scores_to_merge = prepare_player_box_scores(unpacked_player_box_scores)
del(unpacked_player_box_scores)
gc.collect()
train_df = train_df.merge(player_box_scores_to_merge, on=["dailyDataDate", "playerId"], how="left")
del(player_box_scores_to_merge)
gc.collect()

print("# transactions")
unpacked_transactions = unpack_data(train, ["transactions"])["transactions"]
del(train["transactions"])
gc.collect()
transactions_to_merge = prepare_transactions(unpacked_transactions)
del(unpacked_transactions)
gc.collect()
train_df = train_df.merge(transactions_to_merge, on=["dailyDataDate", "playerId"], how="left")
del(transactions_to_merge)
gc.collect()

print("# awards")
unpacked_awards = unpack_data(train, ["awards"])["awards"]
del(train["awards"])
gc.collect()
awards_to_merge = prepare_awards(unpacked_awards)
del(unpacked_awards)
gc.collect()
train_df = train_df.merge(awards_to_merge, on=["dailyDataDate", "playerId"], how="left")
del(awards_to_merge)
gc.collect()

print("# events")
unpacked_events = unpack_data(train, ["events"])["events"]
del(train["events"])
gc.collect()
events_to_merge = prepare_events(unpacked_events, unpacked_games)
del(unpacked_events, unpacked_games)
gc.collect()
train_df = train_df.merge(events_to_merge, on=["dailyDataDate", "playerId"], how="left")
del(events_to_merge)
gc.collect()



print("# standings")
unpacked_standings = unpack_data(train, ["standings"])["standings"]
del(train["standings"])
gc.collect()
standings_to_merge = prepare_standings(unpacked_standings)
del(unpacked_standings)
gc.collect()
train_df = train_df.merge(standings_to_merge, on=["dailyDataDate", "teamId"], how="left")
del(standings_to_merge)
gc.collect()

print("# player twitter followers")
unpacked_player_twitter_followers = unpack_data(train, ["playerTwitterFollowers"])["playerTwitterFollowers"]
del(train["playerTwitterFollowers"])
gc.collect()
train_df = pd.merge_asof(train_df, unpacked_player_twitter_followers.loc[:, ('date', 'playerId', 'numberOfFollowers')].rename(
    columns={"numberOfFollowers": "numberOfFollowersPlayer"}), on="date", by="playerId", direction="backward")
del(unpacked_player_twitter_followers)
gc.collect()

print("# team twitter followers")
unpacked_team_twitter_followers = unpack_data(train, ["teamTwitterFollowers"])["teamTwitterFollowers"]
del(train)
gc.collect()
unpacked_team_twitter_followers["teamId"] = unpacked_team_twitter_followers["teamId"].astype(str)
train_df = pd.merge_asof(train_df, unpacked_team_twitter_followers.loc[:, ("date", "teamId", "numberOfFollowers")].rename(
    columns={"numberOfFollowers": "numberOfFollowersTeam"}), on="date", by="teamId", direction="backward")
del(unpacked_team_twitter_followers)
gc.collect()

print("# mlb social media")
mlb_social_media = pd.read_csv(BASE_DIR / "data/teams.csv")
mlb_social_media = prepare_mlb_social_media(mlb_social_media)
train_df = train_df.merge(mlb_social_media, on="teamId", how="left")
del(mlb_social_media)
gc.collect()

print("# player salaries")
players = pd.read_csv(PATH_TO_MLB_DATA / "players.csv")
player_salaries = pd.read_csv(BASE_DIR / "data/mlbSalaries.csv")
player_salaries_to_merge = prepare_player_salaries(player_salaries, players)
del(player_salaries, players)
gc.collect()
train_df = train_df.merge(player_salaries_to_merge, on=["playerId", "year"], how="left")
del(player_salaries_to_merge)
gc.collect()

train_df.fillna(COLUMN_DEFAULTS, inplace=True)
for feature_name in train_df.columns:
    if feature_name in CATEGORICAL_FEATURE_NAMES:
        train_df[feature_name] = train_df[feature_name].astype(str)
train_df = train_df.loc[:, FEATURE_NAMES + TARGET_FEATURE_NAMES]

# lagged targets
# players
# rosters
# teams
# games
# player box scores
# transactions
# awards
# events


In [ ]:
# use april as holdout
val_df = train_df[((train_df["year"] == "2021") & (train_df["month"] == "4"))]

train_df.drop(val_df.index, inplace=True)
train_df = train_df.sample(frac=1)  # shuffle
gc.collect()

20

In [ ]:
# precompute mean and variance for the normalization layer
NUMERIC_FEATURES_MEAN = {}
NUMERIC_FEATURES_VARIANCE = {}
for feature_name in NUMERIC_FEATURE_NAMES:
    NUMERIC_FEATURES_MEAN[feature_name] = train_df[feature_name].mean()
    NUMERIC_FEATURES_VARIANCE[feature_name] = train_df[feature_name].var()

In [ ]:
# get vocabulary for categorical features
n = 100
CATEGORICAL_FEATURES_WITH_VOCABULARY = {}
for feature_name in CATEGORICAL_FEATURE_NAMES:
    categories_value_count_greater_n = train_df[feature_name].value_counts().index[train_df[feature_name].value_counts() > n]
    CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name] = list(categories_value_count_greater_n)

In [ ]:
def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        if feature_name in CATEGORICAL_FEATURE_NAMES:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(1, ), dtype=tf.string
            )
        else:
            # numeric and binary features
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(1, ), dtype=tf.float32
            )
    return inputs

In [ ]:
# Feature Encoding
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers.experimental.preprocessing import StringLookup


def encode_numerical_feature(feature, name):
    # Create a Normalization layer for our feature
    normalizer = Normalization(mean=NUMERIC_FEATURES_MEAN[name], variance=NUMERIC_FEATURES_VARIANCE[name])

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


def encode_categorical_feature(feature, name, use_embedding):
    # Create a lookup layer which will turn values into integer indices
    vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[name]
    vocab_size = len(vocabulary)
    lookup = StringLookup(num_oov_indices=1, vocabulary=vocabulary, mask_token=None)
    # Turn the input into integer indices
    encoded_feature = lookup(feature)
    
    if use_embedding:
        # Create an embedding layer with the specified dimensions.
        #embedding_dims = int(6*vocab_size**(1/4))
        embedding_dims = EMBEDDING_DIMS
        embedding = layers.Embedding(
            input_dim=vocab_size+1, output_dim=embedding_dims
        )
        # Convert the index values to embedding representations.
        encoded_feature = embedding(encoded_feature)
        encoded_feature = layers.Reshape((embedding_dims, ))(encoded_feature)
        #encoded_feature = layers.Activation("sigmoid")(encoded_feature)
        
    else:
        # Create one hot layer
        one_hot = layers.experimental.preprocessing.CategoryEncoding(
            max_tokens=vocab_size+1, output_mode="binary"
        )
        # Convert index values to one hot vector.
        encoded_feature = one_hot(encoded_feature)
    return encoded_feature


def encode_inputs(inputs, use_embedding):
    encoded_features = []
    for feature_name in inputs:
        if feature_name in CATEGORICAL_FEATURE_NAMES:
            encoded_feature = encode_categorical_feature(inputs[feature_name], feature_name, use_embedding)
        elif feature_name in NUMERIC_FEATURE_NAMES:
            encoded_feature = encode_numerical_feature(inputs[feature_name], feature_name)
        else:
            # no encoding for binary features
            encoded_feature = inputs[feature_name]
        encoded_features.append(encoded_feature)
    if len(encoded_features) > 1:
        all_features = layers.concatenate(encoded_features, axis=-1)
        return all_features
    return encoded_features[0]

In [ ]:
val_dataset = get_dataset_from_df(val_df, batch_size=BATCH_SIZE)

del(val_df)
gc.collect()

0

In [ ]:
train_dataset = get_dataset_from_df(train_df, batch_size=BATCH_SIZE)

del(train_df)
gc.collect()

0

In [ ]:
def create_baseline_model():
    inputs = create_model_inputs()
    features = encode_inputs(inputs, use_embedding=True)
    features = layers.Dense(1000, activation="relu")(features)
    #features = layers.Dense(64, activation="relu")(features)
    outputs = layers.Dense(units=4)(features)

    early_stopping = keras.callbacks.EarlyStopping(patience=1)
    model = keras.Model(inputs=inputs, outputs=outputs)
    #sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer="adam", loss="MAE")
    return model

In [ ]:
def create_wide_and_deep_model(hidden_units):

    inputs = create_model_inputs()
    wide = encode_inputs(inputs, use_embedding=False)
    wide = layers.BatchNormalization()(wide)

    deep = encode_inputs(inputs, use_embedding=True)
    for units in hidden_units:
        deep = layers.Dense(units)(deep)
        deep = layers.BatchNormalization()(deep)
        deep = layers.ReLU()(deep)
        deep = layers.Dropout(0.5)(deep)

    merged = layers.concatenate([wide, deep])
    outputs = layers.Dense(units=4)(merged)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer="adam", loss="MAE")
    return model

In [ ]:
prev_runs = 0
model_name = f"model_lag1_300bs_v{prev_runs+1}"

In [ ]:
model = create_wide_and_deep_model([1024, 1024, 1024, 1024])
early_stopping = keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)
model.fit(train_dataset, epochs=20, validation_data=val_dataset, callbacks=[early_stopping])
model.save(model_name)

Epoch 1/20
8148/8148 [==============================] - 1099s 133ms/step - loss: 0.8332 - val_loss: 0.9628
Epoch 2/20
8148/8148 [==============================] - 1087s 133ms/step - loss: 0.7173 - val_loss: 0.9530
Epoch 3/20
8148/8148 [==============================] - 1060s 130ms/step - loss: 0.6935 - val_loss: 0.9321
Epoch 4/20
8148/8148 [==============================] - 1120s 137ms/step - loss: 0.6773 - val_loss: 0.9234
Epoch 5/20
8148/8148 [==============================] - 1100s 135ms/step - loss: 0.6639 - val_loss: 0.9245
Epoch 6/20
8148/8148 [==============================] - 1062s 130ms/step - loss: 0.6528 - val_loss: 0.9153
Epoch 7/20
8148/8148 [==============================] - 1090s 134ms/step - loss: 0.6438 - val_loss: 0.9119
Epoch 8/20
8148/8148 [==============================] - 1104s 135ms/step - loss: 0.6369 - val_loss: 0.9189
Epoch 9/20
8148/8148 [==============================] - 1073s 132ms/step - loss: 0.6313 - val_loss: 0.9195
Epoch 10/20
8148/8148 [==============